In [1]:
import torch
from torch.autograd import Variable
from torch import nn
from torch.nn import init
import torch.utils.data as Data
import torch.nn.functional as F
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
import os
import matplotlib.pyplot as plt
import warnings

%matplotlib inline

In [2]:
warnings.simplefilter("ignore")

TrainTestDataDir = '/home/songyue/homeCredit/HomeCreditDefaultRisk/Data/TrainTestData'

In [3]:
BATCH_SIZE = 32
INPUT_SIZE = 221
OUTPUT_SIZE = 2
LR = 0.0005
EPOCHES = 200

In [4]:
class Logistic_Regression(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Logistic_Regression, self).__init__()
        self.logistic = nn.Linear(in_dim, n_class)
    
    def forward(self, x):
        out = self.logistic(x)
        return out
model = Logistic_Regression(220,2)

In [5]:
model = model.cuda()

In [6]:
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(),lr=LR)

In [7]:
def loadData(DataName):
    DataPath = os.path.join(TrainTestDataDir, DataName)
    if not os.path.exists(DataPath):
        print('%s does not exist!' % DataPath)
        return
    OriginData = pd.read_csv(DataPath, index_col=0)
    OriginData = OriginData.sample(frac=1)#.reset_index(drop=True)  # 打乱顺序后返回
    return OriginData

In [8]:
def NormalData(TrainData, TestData):
    # 对一些列的均值大于100的进行归一化处理
    AllData = TrainData.append(TestData)
    scaler = StandardScaler().fit(AllData)
    TrainData = scaler.transform(TrainData)
    return TrainData

In [26]:
def train():
    ModelDir = 'models'
    ModelName = 'train1'
    TrainDataName = ModelName + '.csv'
    TestDataName = 'test.csv'
    TrainData = loadData(TrainDataName)
    TestData = loadData(TestDataName)
    # 更改训练样本的index
    TrainData.set_index('SK_ID_CURR', inplace=True)
    TestData.set_index('SK_ID_CURR', inplace=True)
    TrainTarget = TrainData.TARGET
    TrainData.drop('TARGET', axis=1, inplace=True)
    TrainData = NormalData(TrainData, TestData)
    # 划分训练集和验证集
    X_train, X_val, y_train, y_val = train_test_split(TrainData,
                                                      TrainTarget.values, test_size=0.1,
                                                      random_state=711)
    # 将他们转化为torch tensor
    train_x = torch.from_numpy(X_train).type(torch.FloatTensor)
    train_y = torch.from_numpy(y_train).type(torch.LongTensor)
    val_x = Variable(torch.from_numpy(X_val).type(
        torch.FloatTensor), volatile=True).cuda()
    val_y = Variable(torch.from_numpy(y_val).type(
        torch.LongTensor), volatile=True).cuda()
    
    train_dataset = Data.TensorDataset(train_x, train_y)
    train_loader = Data.DataLoader(dataset=train_dataset,
                                   batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
    
    # 开始训练
    trainlosses = []
    vallosses = []
    rosscores = []
    for epoch in range(EPOCHES):
        trainaccTmp = []
        trainlossTmp = []
        for step, (b_x, b_y) in enumerate(train_loader):
            b_x = Variable(b_x).type(torch.FloatTensor).cuda()
            b_y_ans = Variable(b_y).type(torch.FloatTensor)
            b_y = torch.zeros(b_y_ans.cpu().type(torch.LongTensor).unsqueeze(1).shape[0],2).scatter_(1,b_y_ans.cpu().type(torch.LongTensor).unsqueeze(1),1).cuda()
            out = model(b_x).cuda()
            print(out)
            print(b_y)
            print(torch.max(out,1))
            loss = criterion(torch.max(out,1), b_y_ans)
            
#             print(loss)
            
train()

tensor([[-0.3105,  0.1790],
        [-0.5584,  0.2571],
        [ 0.1614, -0.1861],
        [-0.8288, -0.4781],
        [ 0.1354,  0.2466],
        [ 0.1661,  0.0673],
        [ 1.4951,  0.8405],
        [-0.2260, -0.2616],
        [ 0.9596, -0.0093],
        [ 0.1497,  0.0263],
        [-0.2153, -0.0447],
        [ 0.1837,  0.2987],
        [ 0.0690, -0.2174],
        [ 0.9252,  0.5940],
        [ 0.2108, -0.6449],
        [ 0.0520,  0.6078],
        [ 0.0852, -0.0520],
        [-0.1126, -0.2945],
        [-0.7352,  0.2104],
        [-0.3423, -0.2860],
        [ 1.0846,  0.5209],
        [ 0.0579,  0.5049],
        [-0.3805, -0.4361],
        [ 0.0647, -0.4370],
        [-0.1320,  0.3825],
        [-0.1731,  0.2673],
        [-0.1279, -0.0519],
        [-0.4564, -0.0231],
        [-0.4226, -1.1331],
        [ 1.0501,  0.6704],
        [-0.0482, -1.3414],
        [ 0.0020,  0.1051]], device='cuda:0')
tensor([[ 1.,  0.],
        [ 0.,  1.],
        [ 1.,  0.],
        [ 1.,  0.],
      

TypeError: log_softmax(): argument 'input' (position 1) must be Tensor, not tuple